In [1]:
import os
# 【硬性要求】加载环境变量 (确保可以调用 OpenAI API)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
# 导入官方预构建的“填充文档链”工厂函数
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# ==========================================
# 1. 数据摄取 (Data Ingestion)
# ==========================================
# 目标：两篇关于生成式人工智能 (Generative AI) 的科技新闻
URL1 = "https://techcrunch.com/2024/02/27/microsoft-made-a-16-million-investment-in-mistral-ai/"
URL2 = "https://techcrunch.com/2024/03/28/ai21-labs-new-text-generating-ai-model-is-more-efficient-than-most/"

# WebBaseLoader 支持直接传入 URL 列表进行批量加载
print("1. 正在从 TechCrunch 抓取网页数据...")
loader = WebBaseLoader([URL1, URL2])
data = loader.load()

# 验证我们获取了两个 Document 对象
print(f"✅ 成功加载 {len(data)} 篇新闻文档。\n")

# ==========================================
# 2. 定义提示词与模型 (Prompt & Model Definition)
# ==========================================
# 【科研级保姆注释】
# 注意这里的 {context} 变量。create_stuff_documents_chain 在底层是硬编码去寻找 
# "context" 这个键的。如果你把这里写成 {documents}，链就会报错，除非你进行额外配置。
prompt = ChatPromptTemplate.from_messages(
    [("system", "What models are launched by Mistral and AI21 Labs:\n\n{context}")]
)

# 初始化 LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ==========================================
# 3. 构建并调用填充链 (Build & Invoke the Stuff Chain)
# ==========================================
print("2. 正在构建 Stuff Documents Chain 并进行分析...")
# 一行代码将 LLM 和 Prompt 绑定为文档处理专用链
chain = create_stuff_documents_chain(llm, prompt)

# 打印一下链的结构，你会看到它底层依然是一个巨大的 LCEL RunnableBinding
# 这印证了老师说的：它实际上也是基于 runnable 接口的

if __name__ == "__main__":
    # 执行调用。我们必须将包含 Document 对象的列表 (data) 赋给 "context" 键
    response = chain.invoke({"context": data})
    
    print("=== 🤖 AI 分析结果 ===")
    print(response)
    # 预期结果：AI 会综合两篇文章，指出 Mistral 发布了 Mistral Large，AI21 发布了 Jamba。

USER_AGENT environment variable not set, consider setting it to identify your requests.


1. 正在从 TechCrunch 抓取网页数据...
✅ 成功加载 2 篇新闻文档。

2. 正在构建 Stuff Documents Chain 并进行分析...
=== 🤖 AI 分析结果 ===
Mistral AI has launched Mistral Large, a large language model, and a chat assistant. AI21 Labs has launched Jamba, a text-generating and -analyzing model.
